<a href="https://colab.research.google.com/github/cij1012/pytrorch/blob/master/%EC%86%8C%ED%94%84%ED%8A%B8%EB%A7%A5%EC%8A%A4%20%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

소프트맥스 회귀의 비용함수 구현하기

In [0]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [0]:
z = torch.FloatTensor([1,2,3])

In [0]:
hypothesis = F.softmax(z, dim=0)
hypothesis

tensor([0.0900, 0.2447, 0.6652])

In [0]:
hypothesis.sum()

tensor(1.)

In [0]:
z = torch.rand(3, 5, requires_grad=True)
z

tensor([[0.4550, 0.5725, 0.4980, 0.9371, 0.6556],
        [0.3138, 0.1980, 0.4162, 0.2843, 0.3398],
        [0.5239, 0.7981, 0.7718, 0.0112, 0.8100]], requires_grad=True)

In [0]:
hypothesis = F.softmax(z, dim = 1)
hypothesis

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)

In [0]:
y = torch.randint(5, (3,)).long()
y

tensor([3, 1, 2])

In [0]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [0]:
y.unsqueeze(1)

tensor([[3],
        [1],
        [2]])

In [0]:
y_one_hot

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [0]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward>)

In [0]:
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward>)

In [0]:
#1
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4992, grad_fn=<MeanBackward0>)

In [0]:
#2
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4992, grad_fn=<MeanBackward0>)

In [0]:
#3
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4992, grad_fn=<NllLossBackward>)

In [0]:
#4
F.cross_entropy(z, y)

tensor(1.4992, grad_fn=<NllLossBackward>)

소프트맥스 회귀 구현하기

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

로우레벨 소프트맥스 회귀 구현하기

In [0]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [0]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
y_one_hot.shape

torch.Size([8, 3])

In [0]:
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(3, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

In [0]:

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


하이레벨 소프트맥스 회귀

In [0]:
W = torch.zeros((4,3),requires_grad=True)
b = torch.zeros(3,requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

  z = x_train.matmul(W)+b
  cost = F.cross_entropy(z, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100==0:
    print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.761050
Epoch  200/1000 Cost : 0.689991
Epoch  300/1000 Cost : 0.643229
Epoch  400/1000 Cost : 0.604117
Epoch  500/1000 Cost : 0.568255
Epoch  600/1000 Cost : 0.533922
Epoch  700/1000 Cost : 0.500291
Epoch  800/1000 Cost : 0.466908
Epoch  900/1000 Cost : 0.433506
Epoch 1000/1000 Cost : 0.399962


소프트맥스 회귀 nn.Module로 구현

In [0]:
model = nn.Linear(4,3)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
  prediction = model(x_train)
  cost = F.cross_entropy(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 ==0:
    print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch,nb_epochs,cost.item()))

Epoch    0/1000 Cost : 1.845720
Epoch  100/1000 Cost : 0.647150
Epoch  200/1000 Cost : 0.568868
Epoch  300/1000 Cost : 0.515699
Epoch  400/1000 Cost : 0.471727
Epoch  500/1000 Cost : 0.432486
Epoch  600/1000 Cost : 0.395879
Epoch  700/1000 Cost : 0.360506
Epoch  800/1000 Cost : 0.325227
Epoch  900/1000 Cost : 0.289217
Epoch 1000/1000 Cost : 0.254086


소프트맥스 회귀 클래스로 구현

In [0]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)
  def forward(self, x):
    return self.linear(x)

In [0]:
model = SoftmaxClassifierModel()

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 3.586500
Epoch  100/1000 Cost: 0.650367
Epoch  200/1000 Cost: 0.566842
Epoch  300/1000 Cost: 0.512495
Epoch  400/1000 Cost: 0.468165
Epoch  500/1000 Cost: 0.428759
Epoch  600/1000 Cost: 0.391982
Epoch  700/1000 Cost: 0.356357
Epoch  800/1000 Cost: 0.320726
Epoch  900/1000 Cost: 0.284398
Epoch 1000/1000 Cost: 0.250610


소프트맥스 회귀로 MNIST 데이터 분류하기